In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
from tqdm import tqdm

In [2]:
def calculate_percent_positive(df, channel, threshold=3):
    # initialize variables
    positive = {}
    positive_mean = {}
    positive_std = {}

    x = np.linspace(300,1500,100)
    
    # iterate through each siRNA
    for target in tqdm(df['siRNA target'].unique()):
        # initialize variables
        positive[target] = {}
        positive_mean[target] = {}
        positive_std[target] = {}
        count = 0
        target_df = df[df['siRNA target']==target]
        fig = plt.figure(figsize=(20,15))
        
        # iterate through each replicate
        for replicate in sorted(target_df['Replicate'].unique()):
            rep_df = target_df[target_df['Replicate']==replicate]

            # iterate through each field of each replicate
            for field in range(1,5):
                field_df = rep_df[rep_df['Field']==field]
                
                # fit background distribution
                values = field_df[channel]
                median = np.median(values)
                reflected = values[values<median]
                reflected = np.hstack([2*median-reflected, reflected])
                loc,scale = stats.norm.fit(reflected)
                
                bg = values[values<(loc+scale*2)]
                m,s = stats.norm.fit(bg)
                
                # calculate GFP positive
                positive[target]['%i.%i' % (replicate, field)] = np.count_nonzero(values>(m+threshold*s))/np.count_nonzero(values)

                # plot distributions and save to file
                count += 1
                ax = fig.add_subplot(3,4,count)
                ax.hist(values, bins=x, fc='silver')
                ax.set_title('Replicate %i, Field %i' % (replicate,field), fontsize=18)
                ax2 = ax.twinx()
                ax2.set_yticks([])
                ax2.plot(x,stats.norm.pdf(x,m,s),c='r')
                ax.axvline(m+threshold*s,c='k')
                bottom,top = ax2.get_ylim()
                ax2.set_ylim(0,top)
                fig.supylabel('# of cells', fontsize=18)
                fig.supxlabel(channel, fontsize=18)
                fig.tight_layout()
                fluor = channel.split(' ')[0]
                fig.savefig('%s_%s.png' % (fluor,target))
                plt.close()
                
                
            # calculate means and standard deviations
            positive_mean[target][replicate] = np.mean([positive[target]['%i.%i' % (replicate,x)] for x in range(1,5)])
            positive_std[target][replicate] = np.std([positive[target]['%i.%i' % (replicate,x)] for x in range(1,5)])
            

    return positive_mean, positive_std

In [ ]:
# read in data
rep1 = pd.read_csv('2023.8.4 S1P1R1 objects.csv')
rep2 = pd.read_csv('2023.8.4 S1P1R2 objects.csv')
rep3 = pd.read_csv('2023.8.4 S1P1R3 objects.csv')

data = pd.concat([rep1,rep2,rep3]).reset_index(drop=True)

In [10]:
# calculate % GFP positive

GFP_mean, GFP_std = calculate_percent_positive(data,'GFP intensity')

GFP_df = pd.merge(pd.DataFrame.from_records([(level1, level2, leaf)
                                             for level1, level2_dict in GFP_mean.items()
                                             for level2, leaf in level2_dict.items()],
                                            columns=['target', 'replicate', 'GFP_mean']),
                  pd.DataFrame.from_records([(level1, level2, leaf)
                                             for level1, level2_dict in GFP_std.items()
                                             for level2, leaf in level2_dict.items()],
                                            columns=['target', 'replicate', 'GFP_std']))


In [11]:
# calculate % mCherry positive

mCherry_mean, mCherry_std = calculate_percent_positive(data,'mCherry intensity')

mCherry_df = pd.merge(pd.DataFrame.from_records([(level1, level2, leaf)
                                             for level1, level2_dict in mCherry_mean.items()
                                             for level2, leaf in level2_dict.items()],
                                            columns=['target', 'replicate', 'mCherry_mean']),
                  pd.DataFrame.from_records([(level1, level2, leaf)
                                             for level1, level2_dict in mCherry_std.items()
                                             for level2, leaf in level2_dict.items()],
                                            columns=['target', 'replicate', 'mCherry_std']))

In [12]:
# merge dataframes and export to file

out_df = pd.merge(GFP_df,mCherry_df)
out_df.to_csv('out.csv',index=False)